In [1]:
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
from asgiref.sync import sync_to_async


In [3]:

import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "orm.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

from src.postprocess.postprocess import *
from src.models import *


In [4]:
results = await sync_to_async(Bench.objects.filter(name="mpp-varda-one-jvm-tracing").order_by('-pk')[0].results.all)()
results[0].results.keys()

odict_keys(['duration'])

In [5]:
def pad_dict_list(dict_list, padel):
    lmax = 0
    for lname in dict_list.keys():
        lmax = max(lmax, len(dict_list[lname]))
    for lname in dict_list.keys():
        ll = len(dict_list[lname])
        if  ll < lmax:
            dict_list[lname] += [padel] * (lmax - ll)
    return dict_list

In [6]:
def get_traces_df(prefix, n):
    #stats1 = Stats([results[0]]).extract(
    #    "n", "durationsReception2OnMessage", lambda x: True, "")
    stats2 = Stats([results[0]]).extract(
        "n", f"{prefix}durationsReception2Dispatcher", lambda x: True, "")
    stats3 = Stats([results[0]]).extract(
        "n", f"{prefix}durationsReception2Callback", lambda x: True, "")
    stats4 = Stats([results[0]]).extract(
        "n", f"{prefix}durationsReception2EndCallback", lambda x: True, "")
    stats5 = Stats([results[0]]).extract(
        "n", f"{prefix}durationsPing", lambda x: True, "")
    stats6 = Stats([results[0]]).extract(
        "n", f"{prefix}durationsPingAfterSessionInit", lambda x: True, "")
    stats7 = Stats([results[0]]).extract(
        "n", f"{prefix}durationsPingAfterComputeSize", lambda x: True, "")

    duration = int(results[0].results["duration"]["value"])

    df2 = pd.DataFrame(
        pad_dict_list({   
            #"durationsReception2OnMessage": stats1[n].items, 
            "durationsReception2Dispatcher": stats2[n].items,
            "durationsReception2Callback": stats3[n].items,  
            "durationsReception2EndCallback": stats4[n].items,
            "durationsPing": stats5[n].items,  
            "durationsPingAfterSessionInit": stats6[n].items,  
            "durationsPingAfterComputeSize": stats7[n].items,  
        }, -1))
    df2 = df2/1000000   #conver ns -> ms
    df2["rtt"] = Stats([results[0]]).extract(
        "n", "rtt", lambda x: True, "")[n].items
    df2["duration"] = [duration for _ in range(n)] 
    return df2 

In [7]:
warmup = results[0].run_config['warmup']
n = results[0].run_config['n']
(warmup, n)


(0, 100)

In [8]:
duration = int(results[0].results["duration"]["value"])
duration

ValueError: invalid literal for int() with base 10: 'N/A'

In [ ]:
def get_network_df(prefix, n):
    stats1 = Stats([results[0]]).extract(
        "n", f"{prefix}durationsFire", lambda x: True, "")
    stats2 = Stats([results[0]]).extract(
        "n", f"{prefix}durationsFireAfterChecks", lambda x: True, "")
    stats3 = Stats([results[0]]).extract(
        "n", f"{prefix}durationsFireAfterHydratation", lambda x: True, "")
    stats4 = Stats([results[0]]).extract(
        "n", f"{prefix}durationsFireAfterTell", lambda x: True, "")

    df2 = pd.DataFrame(
        pad_dict_list({   
            "durationsFire": stats1[n].items, 
            "durationsFireAfterChecks": stats2[n].items,
            "durationsFireAfterHydratation": stats3[n].items,
            "durationsFireAfterTell": stats4[n].items,
        },-1))
    df2 = df2/1000000   #conver ns -> ms
    #df2 = df2.drop([0,1]) #get ride of the first two lines
    return df2 

In [ ]:
df_ping = get_traces_df("ping", n)
df_pong = get_traces_df("pong", n)
df_ping_session = get_network_df("ping", n)
df_pong_session = get_network_df("pong", n)


In [ ]:
df_pong.plot(
    title="Tracing of the Pong actor",
    y=[
        "durationsReception2Dispatcher", 
        "durationsReception2Callback",
        "durationsReception2EndCallback",
        #"durationsReception2OnMessage"
    ],
    ylabel="duration (ms)",
    xlabel="message id"
).get_figure().show()  # savefig("test-hist.png")

In [ ]:
df_pong_session.plot(
    title="Session statistics of the Pong actor",
    y=[
        "durationsFire",
        "durationsFireAfterChecks",
        "durationsFireAfterHydratation",
        "durationsFireAfterTell",
    ],
    ylabel="duration (ms)",
    xlabel="message id"
).get_figure().show()

In [ ]:
df_ping_session.plot(
    title="Session statistics of the Ping actor",
    y=[
        "durationsFire",
        "durationsFireAfterChecks",
        "durationsFireAfterHydratation",
        "durationsFireAfterTell",
    ],
    ylabel="duration (ms)",
    xlabel="message id"
).get_figure().show()

In [ ]:
df_ping.plot(
    title="Tracing of the Ping actor",
    y=[
        "durationsReception2Dispatcher", 
        "durationsReception2Callback",
        "durationsReception2EndCallback",
        "durationsPing",
        "durationsPingAfterSessionInit",
        "durationsPingAfterComputeSize",
        "rtt",
        "duration" 
        #"durationsReception2OnMessage"
    ],
    ylabel="duration (ms)",
    xlabel="message id"
).get_figure().show()  # savefig("test-hist.png")
